# PriceTrack: Unlocking Car Market Insights

PriceTrack is a data science project designed to predict the valuation of second-hand cars based on key input parameters. 
Leveraging regression model, it provides data-driven insights to help buyers and sellers make informed decisions.

## Developing Regression Model

Importing all the required modules and functions

In [299]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

Loading the cleaned data

In [300]:
df = pd.read_csv("cleaned_data.csv")
df

,name,company,year,Price,kms_driven,fuel_type,Age,Annual_Km_Driven
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,80000,45000,Petrol,13,3461.538462
1,Mahindra Jeep CL550 MDI,Mahindra,2006,425000,40,Diesel,14,2.857143
2,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,325000,28000,Petrol,6,4666.666667
3,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,575000,36000,Diesel,6,6000.000000
4,Ford Figo,Ford,2012,175000,41000,Diesel,8,5125.000000
...,...,...,...,...,...,...,...,...
811,Maruti Suzuki Ritz VXI ABS,Maruti,2011,270000,50000,Petrol,9,5555.555556
812,Tata Indica V2 DLE BS III,Tata,2009,110000,30000,Diesel,11,2727.272727
813,Toyota Corolla Altis,Toyota,2009,300000,132000,Petrol,11,12000.000000
814,Tata Zest XM Diesel,Tata,2018,260000,27000,Diesel,2,13500.000000


### Feature Engineering: Age & Kilometers Driven Transformation  

- **`Age` Calculation:**  
  The car's age is computed as:  
  $$
  \text{Age} = \text{Max Year} - \text{Car's Year} + 1
  $$
  - Newer cars have a **lower age value**, while older cars have a **higher age value** to reflect depreciation.

- **`kms_driven` Inversion:**  
  The **`kms_driven`** column is transformed using:  
  $$
  \text{kms\_driven} = \text{Max kms\_driven} - \text{kms\_driven}
  $$  
  - Higher mileage cars get **lower values** (indicating more wear).  
  - Lower mileage cars get **higher values** (indicating better condition).  

- The original **`year`** column is removed, as it's no longer needed after transformation.  

This transformation improves the model’s ability to capture **price depreciation** over time and with increased usage. 🚗📉  


In [301]:
useful_df = df.copy()
useful_df["Age"] = df["year"].max() - df["year"] + 1  # Assume max year as base year
useful_df = useful_df.drop(columns=["year"])  # Drop original year column
useful_df["kms_driven"] = df["kms_driven"].max() - df["kms_driven"] # Invert kms_driven

# # Replace less frequent companies with "Other"
# top_companies = useful_df["company"].value_counts().nlargest(4).index  # Top 4 companies
# useful_df["company"] = useful_df["company"].apply(lambda x: x if x in top_companies else "Other")

useful_df

,name,company,Price,kms_driven,fuel_type,Age,Annual_Km_Driven
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,80000,155000,Petrol,13,3461.538462
1,Mahindra Jeep CL550 MDI,Mahindra,425000,199960,Diesel,14,2.857143
2,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,325000,172000,Petrol,6,4666.666667
3,Ford EcoSport Titanium 1.5L TDCi,Ford,575000,164000,Diesel,6,6000.000000
4,Ford Figo,Ford,175000,159000,Diesel,8,5125.000000
...,...,...,...,...,...,...,...
811,Maruti Suzuki Ritz VXI ABS,Maruti,270000,150000,Petrol,9,5555.555556
812,Tata Indica V2 DLE BS III,Tata,110000,170000,Diesel,11,2727.272727
813,Toyota Corolla Altis,Toyota,300000,68000,Petrol,11,12000.000000
814,Tata Zest XM Diesel,Tata,260000,173000,Diesel,2,13500.000000


### Defining Features and Target Variable

- **`X` (Features)**: The independent variables used for prediction:
  - `company`: The brand of the car (categorical).
  - `fuel_type`: The type of fuel the car uses (categorical).
  - `kms_driven`: The total distance the car has been driven (numerical).
  - `Age`: The age of the car, derived from the manufacturing year (numerical).

- **`y` (Target Variable)**:  
  - `Price`: The dependent variable representing the car's selling price.

This selection ensures that the model learns the relationship between these key factors and the car's valuation.


In [302]:
X = useful_df[["company", "fuel_type", "kms_driven", "Age"]]
y = useful_df["Price"]

### Train-Test Split

- The dataset is split into **training** and **testing** sets to evaluate model performance.
- **`train_test_split(X, y, test_size=0.2, random_state=42)`**:
  - **80%** of the data is used for training (`X_train`, `y_train`).
  - **20%** of the data is reserved for testing (`X_test`, `y_test`).
  - `random_state=42` ensures reproducibility by generating the same split every time.

This helps in assessing how well the model generalizes to unseen data.


In [303]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Data Preprocessing Pipeline

The `preprocessor` is a `ColumnTransformer` that applies different transformations to categorical and numerical features:

- **One-Hot Encoding (`OneHotEncoder`)**: Converts categorical variables (`company`, `fuel_type`) into numerical format while ignoring unknown categories.
- **Feature Scaling (`StandardScaler`)**: Standardizes numerical features (`kms_driven`, `Age`) to improve model performance by ensuring they have a mean of 0 and a standard deviation of 1.

This preprocessing step ensures that both categorical and numerical data are properly formatted before feeding them into the regression model.


In [304]:
preprocessor = ColumnTransformer([
    ("onehot", OneHotEncoder(handle_unknown='ignore'), ["company", "fuel_type"]),
    ("scaler", StandardScaler(), ["kms_driven", "Age"])
])
preprocessor

ColumnTransformer(transformers=[('onehot',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['company', 'fuel_type']),
                                ('scaler', StandardScaler(),
                                 ['kms_driven', 'Age'])])

### Model Pipeline

A **`Pipeline`** is used to streamline preprocessing and model training in a single workflow:

- **`preprocessor`**: Applies transformations to the input data (One-Hot Encoding & Standard Scaling).
- **`LinearRegression()`**: The regression model that learns the relationship between the features and the target variable.

This approach ensures that preprocessing steps are consistently applied during both training and prediction, improving efficiency and reducing the risk of data leakage.


In [305]:
model = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

### Training & Predictions

- **`model.fit(X_train, y_train)`**: Trains the Linear Regression model using the training dataset.  
- **`y_pred = model.predict(X_test)`**: Predicts car prices on the test dataset for evaluation.








In [306]:
# Train Model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

### Model Evaluation Metrics

- **Mean Absolute Error (MAE)**: Measures the average absolute difference between actual and predicted prices. Lower values indicate better accuracy.
- **Mean Squared Error (MSE)**: Similar to MAE but gives higher weight to larger errors, making it more sensitive to outliers.
- **Root Mean Squared Error (RMSE)**: Square root of MSE, providing an interpretable error measure in the same unit as price.
- **R² Score**: Indicates how well the model explains the variance in price; closer to 1 means a better fit.

In [307]:
# Model Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Display Metrics
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R2 Score: {r2:.2f}")

MAE: 155791.28
MSE: 72706443291.23
RMSE: 269641.32
R2 Score: 0.56


Checking the co-efficients

In [308]:
feature_names = preprocessor.get_feature_names_out()
coef_df = pd.DataFrame(model.named_steps["regressor"].coef_, index=feature_names, columns=["Coefficient"])
coef_df

,Coefficient
onehot__company_Audi,6.989568e+05
onehot__company_BMW,2.239913e+05
onehot__company_Chevrolet,-5.463950e+05
onehot__company_Datsun,-5.618698e+05
onehot__company_Fiat,-5.114058e+05
onehot__company_Force,-2.802534e+05
onehot__company_Ford,-3.678633e+05
onehot__company_Hindustan,-5.864726e+04
onehot__company_Honda,-3.743896e+05
onehot__company_Hyundai,-3.926298e+05


### Model Testing: Manual Test Cases  

To validate the accuracy and behavior of the trained regression model, we run manual test cases with different car attributes. These test cases help check if the model is making reasonable predictions based on the expected trends.  

| **Company** | **Fuel Type** | **Kilometers Driven** | **Age (Years)** | **Description** | **Expected Outcome** |
|------------|-------------|-----------------|-------------|----------------|----------------|
| Toyota     | Petrol      | 1,000           | 1           | New Car with Low Mileage | A high predicted price, as the car is new with low mileage. |
| Honda      | Diesel      | 200,000         | 10          | Old Car with High Mileage | A significantly lower predicted price, as the car has high mileage and is very old. |
| BMW        | Petrol      | 20,000          | 2           | Luxury Car with Low Mileage | A relatively high predicted price, as luxury cars tend to retain value better. |
| Hyundai    | Petrol      | 50,000          | 5           | Mid-range Car with Moderate Usage | A moderate price, considering the car is neither too old nor too new. |
| Maruti     | Petrol      | 150,000         | 8           | Economy Car with High Mileage | A lower predicted price due to high mileage and moderate age. |

### **Key Trends Expected**  
- **Newer cars with lower mileage** should have a **higher price**.  
- **Older cars with high mileage** should have a **lower price**.  
- **Luxury brands (BMW, Jaguar, Mercedes, etc.)** should have a **higher predicted price** compared to economy brands.  
- **Diesel cars** might have a **slightly higher price** than petrol cars due to fuel efficiency.  

These test cases ensure that the model adheres to logical pricing patterns based on mileage, age, and brand value. 🚗💨


In [309]:
# Define test cases as a list of dictionaries
test_cases = [
    { "company": "Toyota", "fuel_type": "Petrol", "kms_driven": 1000, "Age": 1, "Description": "New Car with Low Mileage",},
    { "company": "Honda", "fuel_type": "Diesel", "kms_driven": 200000, "Age": 10, "Description": "Old Car with High Mileage",},
    { "company": "BMW", "fuel_type": "Petrol", "kms_driven": 20000, "Age": 2, "Description": "Luxury Car with Low Mileage",},
    { "company": "Hyundai", "fuel_type": "Petrol", "kms_driven": 50000, "Age": 5, "Description": "Mid-range Car with Moderate Usage"},
    { "company": "Maruti", "fuel_type": "Petrol", "kms_driven": 150000, "Age": 8, "Description": "Economy Car with High Mileage",}
]

# Convert test cases to DataFrame
test_df = pd.DataFrame(test_cases)

# Predict prices for test cases
predicted_prices = model.predict(test_df.drop(columns=["Description"]))

# Add predicted prices to DataFrame
test_df["Predicted Price (₹)"] = [f"₹{price:,.0f}" for price in predicted_prices]

# Display the results in a structured table
test_df

,company,fuel_type,kms_driven,Age,Description,Predicted Price (₹)
0,Toyota,Petrol,1000,1,New Car with Low Mileage,"₹735,638"
1,Honda,Diesel,200000,10,Old Car with High Mileage,"₹351,279"
2,BMW,Petrol,20000,2,Luxury Car with Low Mileage,"₹1,079,182"
3,Hyundai,Petrol,50000,5,Mid-range Car with Moderate Usage,"₹373,019"
4,Maruti,Petrol,150000,8,Economy Car with High Mileage,"₹249,869"
